### Dependencies

In [90]:
import os
os.environ['PYARROW_IGNORE_TIMEZONE'] = '1'

import numpy as np
from pyspark import RDD
from pyspark.sql import SparkSession
from pyspark.sql.types import TimestampType
import pyspark.pandas as ps
import io
import requests


25/02/03 17:12:24 ERROR RetryingBlockTransferor: Exception while beginning fetch of 1 outstanding blocks (after 2 retries)
java.io.IOException: Connecting to /192.168.1.86:52245 failed in the last 4750 ms, fail this connection directly
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:210)
	at org.apache.spark.network.netty.NettyBlockTransferService$$anon$2.createAndStart(NettyBlockTransferService.scala:131)
	at org.apache.spark.network.shuffle.RetryingBlockTransferor.transferAllOutstanding(RetryingBlockTransferor.java:173)
	at org.apache.spark.network.shuffle.RetryingBlockTransferor.lambda$initiateRetry$0(RetryingBlockTransferor.java:206)
	at java.base/java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:515)
	at java.base/java.util.concurrent.FutureTask.run(FutureTask.java:264)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecut

### Display Options

## Spark Session

In [27]:
ps.options.display.max_rows = 10
# 공식 문서: compute.max_rows
# 1000행 이하라면 driver로 데이터를 가져와서 pandas API로 처리.
# 1000행 이상이면 pySpark로 처리
print(ps.options.compute.max_rows)

1000


## Spark Configuration

In [92]:
# spark.stop()
spark = SparkSession.builder \
        .appName("test") \
        .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
        .config("spark.driver.memory", "6g") \
        .config("spark.executor.memory", "6g") \
        .config("spark.driver.maxResultSize", "4g") \
        .config("spark.sql.shuffle.partitions", "100") \
        .config('spark.driver.bindAddress', '127.0.0.1') \
        .getOrCreate()
        
# 1. 모든 설정 확인
all_configs = spark.sparkContext.getConf().getAll()
for key, value in all_configs:
    print(f"{key}: {value}")

# # 3. SQL 관련 설정만 확인
# sql_configs = spark.sql("SET -v").collect()
# for row in sql_configs:
#     print(f"{row['key']}: {row['value']}")

# # 4. Runtime에 설정된 값들 확인
# runtime_conf = spark.sparkContext._conf.getAll()
# for key, value in runtime_conf:
#     print(f"{key}: {value}")

spark.app.id: local-1738570418598
spark.driver.extraJavaOptions: -Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false
spark.app.name: test
spark.app.submitTime: 1738562721766
spark.executor.id: dri

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 54128)
Traceback (most recent call last):
  File "/Users/admin/.pyenv/versions/3.10.12/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/admin/.pyenv/versions/3.10.12/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Users/admin/.pyenv/versions/3.10.12/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/admin/.pyenv/versions/3.10.12/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/Users/admin/workspace/python/DE/.venv/lib/python3.10/site-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/Users/admin/workspace/python/DE/.venv/lib/python3.10/site-packages/pyspark/accumulators.py

### Data Loading

In [93]:
# DataFrame으로 읽기
psdf = spark.read.parquet("materials/data_engineering_course_materials/missions/W4/tlc/yellow_tripdata_2024-02.parquet")

rdd = psdf.rdd

In [43]:
rdd.take(3)

[Row(VendorID=2, tpep_pickup_datetime=datetime.datetime(2024, 2, 1, 0, 4, 45), tpep_dropoff_datetime=datetime.datetime(2024, 2, 1, 0, 19, 58), passenger_count=1, trip_distance=4.39, RatecodeID=1, store_and_fwd_flag='N', PULocationID=68, DOLocationID=236, payment_type=1, fare_amount=20.5, extra=1.0, mta_tax=0.5, tip_amount=1.28, tolls_amount=0.0, improvement_surcharge=1.0, total_amount=26.78, congestion_surcharge=2.5, Airport_fee=0.0),
 Row(VendorID=2, tpep_pickup_datetime=datetime.datetime(2024, 2, 1, 0, 56, 31), tpep_dropoff_datetime=datetime.datetime(2024, 2, 1, 1, 10, 53), passenger_count=1, trip_distance=7.71, RatecodeID=1, store_and_fwd_flag='N', PULocationID=48, DOLocationID=243, payment_type=1, fare_amount=31.0, extra=1.0, mta_tax=0.5, tip_amount=9.0, tolls_amount=0.0, improvement_surcharge=1.0, total_amount=45.0, congestion_surcharge=2.5, Airport_fee=0.0),
 Row(VendorID=2, tpep_pickup_datetime=datetime.datetime(2024, 2, 1, 0, 7, 50), tpep_dropoff_datetime=datetime.datetime(2024

In [94]:
num_partitions = rdd.getNumPartitions()
print(f"Number of partitions: {num_partitions}")

Number of partitions: 10


## Data Cleaning

### Narrow Transformations

In [95]:
from datetime import datetime
start_date = datetime.strptime("2024-02-01", "%Y-%m-%d")
end_date = datetime.strptime("2024-03-01", "%Y-%m-%d")

filtered_rdd = rdd.filter(lambda x: x['VendorID'] != None) \
    .filter(lambda x: x['tpep_pickup_datetime'] != None) \
    .filter(lambda x: start_date <= x['tpep_pickup_datetime'] < end_date) \
    .filter(lambda x: x['tpep_dropoff_datetime'] != None) \
    .filter(lambda x: start_date <= x['tpep_dropoff_datetime'] < end_date) \
    .filter(lambda x: x['passenger_count'] != None ) \
    .filter(lambda x: x['passenger_count'] > 0) \
    .filter(lambda x: x['trip_distance'] != None ) \
    .filter(lambda x: x['trip_distance'] > 0) \
    .filter(lambda x: x['RatecodeID'] != None ) \
    .filter(lambda x: x['store_and_fwd_flag'] != None ) \
    .filter(lambda x: x['PULocationID'] != None ) \
    .filter(lambda x: x['DOLocationID'] != None ) \
    .filter(lambda x: x['payment_type'] != None ) \
    .filter(lambda x: x['fare_amount'] != None ) \
    .filter(lambda x: x['fare_amount'] > 0 ) \
    .filter(lambda x: x['extra'] != None ) \
    .filter(lambda x: x['mta_tax'] != None ) \
    .filter(lambda x: x['tip_amount'] != None ) \
    .filter(lambda x: x['tolls_amount'] != None ) \
    .filter(lambda x: x['improvement_surcharge'] != None ) \
    .filter(lambda x: x['total_amount'] != None ) \
    .filter(lambda x: x['total_amount'] > 0 ) \
    .filter(lambda x: x['congestion_surcharge'] != None ) \
    .filter(lambda x: x['Airport_fee'] != None )

In [130]:
filtered_rdd.cache()
filtered_rdd.count()

Py4JJavaError: An error occurred while calling o2676.checkpoint.
: org.apache.spark.SparkException: Checkpoint directory has not been set in the SparkContext
	at org.apache.spark.errors.SparkCoreErrors$.checkpointDirectoryHasNotBeenSetInSparkContextError(SparkCoreErrors.scala:160)
	at org.apache.spark.rdd.RDD.checkpoint(RDD.scala:1660)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:

In [128]:
# 놓친 것이 있는지 확인
filtered_rdd.filter(lambda x: x['RatecodeID'] < 0 ) \
            .filter(lambda x: x['PULocationID'] < 0 ) \
            .filter(lambda x: x['DOLocationID'] < 0 ) \
            .filter(lambda x: x['payment_type'] < 0 ) \
            .filter(lambda x: x['fare_amount'] < 0 ) \
            .filter(lambda x: x['extra'] < 0 ) \
            .filter(lambda x: x['mta_tax'] < 0 ) \
            .filter(lambda x: x['tip_amount'] < 0 ) \
            .filter(lambda x: x['tolls_amount'] < 0 ) \
            .filter(lambda x: x['improvement_surcharge'] < 0 ) \
            .filter(lambda x: x['total_amount'] < 0 ) \
            .filter(lambda x: x['congestion_surcharge'] < 0 ) \
            .filter(lambda x: x['Airport_fee'] < 0 ) \
            .count()

0

## Transformation Logic:


In [131]:
# 총 매출
# 방법 1: map, sum 사용
tot_revenue = filtered_rdd.map(lambda x: x['total_amount']).sum()
print(f"Total revenue: {tot_revenue}")

# 방법 2: reduce 사용
tot_revenue = filtered_rdd.map(lambda x: x['total_amount']).reduce(lambda x, y: x + y)
print(f"Total revenue: {tot_revenue}")
# 방법 3: fold 사용 (초기 시작값이 있음)
tot_revenue = filtered_rdd.map(lambda x: x['total_amount']).fold(0, lambda acc, x: acc + x)
print(f"Total revenue: {tot_revenue}") 

Total revenue: 74431279.74007908


Total revenue: 74431279.74007908


Total revenue: 74431279.74007908


AttributeError: 'float' object has no attribute 'checkpoint'

25/02/03 21:18:58 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1056436 ms exceeds timeout 120000 ms
25/02/03 21:18:58 WARN SparkContext: Killing executors is not supported by current scheduler.
25/02/03 21:35:46 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$

In [78]:
# 총 trip 수
tot_trips = filtered_rdd.count()
print(f"Total trips: {tot_trips}")

Total trips: 2718651


## Aggregate

In [79]:
stats = filtered_rdd.aggregate(
    (0, 0.0, 0), # 초기값 (trip 수, 총 매출, 평균 거리)
    # combine value
    # # 각 partition에서 수행할 연산 (map)
    lambda acc, x: (
        acc[0] +1, 
        acc[1] + x['total_amount'],
        acc[2] + x['trip_distance'],
        ),
    # 각 partition 결과를 합치는 연산 (combine combiners)
    lambda acc1, acc2: ( # 
        acc1[0] + acc2[0], 
        acc1[1] + acc2[1],
        acc1[2] + acc2[2],        
        ),     
)

print(f"Total trips: {stats[0]}")
print(f"Total revenue: {stats[1]}")
print(f"Average distance: {stats[2] / stats[0]}")


Total trips: 2718651
Total revenue: 74431279.74007908
Average distance: 3.419221959714512


In [99]:
# 1. map, reduceByKey, sortByKey 사용
# number of trips per day
daily_trips = filtered_rdd.map(
    lambda x: (x['tpep_pickup_datetime'].date(), 1)
).reduceByKey(lambda x, y: x + y).sortByKey()

for date, count in daily_trips.collect():
    print(f"{date}: {count}")

# 2. groupByKey 사용 (누적하는 것이 아닌 한 번에 메모리에 올리기 때문에 메모리 부족할 수 있음)
# 2m 30s
# number of trips per day
# daily_trips = filtered_rdd.keyBy(
#     lambda x: x['tpep_pickup_datetime'].strftime('%Y-%m-%d')
# ).groupByKey().mapValues(
#     lambda x: len(list(x))
# ).collect()

# for date, count in daily_trips:
#     print(f"{date}: {count}")

# 3. Join도 가능하지만 Inner 값이기에 비효율적

2024-02-01: 102070
2024-02-02: 98021
2024-02-03: 100990
2024-02-04: 80983
2024-02-05: 82904
2024-02-06: 95328
2024-02-07: 95906
2024-02-08: 105306
2024-02-09: 101060
2024-02-10: 102715
2024-02-11: 85507
2024-02-12: 85609
2024-02-13: 64354
2024-02-14: 108629
2024-02-15: 107069
2024-02-16: 94776
2024-02-17: 91075
2024-02-18: 80718
2024-02-19: 70098
2024-02-20: 90266
2024-02-21: 96718
2024-02-22: 105507
2024-02-23: 94591
2024-02-24: 103426
2024-02-25: 82841
2024-02-26: 79291
2024-02-27: 100885
2024-02-28: 100272
2024-02-29: 111736


In [100]:
# Total revenue per day
daily_revenue = filtered_rdd.map(
    lambda x: (x['tpep_pickup_datetime'].date(), x['total_amount'])
).reduceByKey(lambda x, y: x + y).sortByKey()

for date, revenue in daily_revenue.collect():
    print(f"{date}: {revenue}")

2024-02-01: 2824015.919999942
2024-02-02: 2620733.9299999326
2024-02-03: 2491385.499999978
2024-02-04: 2237400.2499998766
2024-02-05: 2316529.6799999434
2024-02-06: 2580443.3999999897
2024-02-07: 2696189.720000007
2024-02-08: 2960078.48999986
2024-02-09: 2804011.5299998904
2024-02-10: 2608000.2499999367
2024-02-11: 2330352.6399999503
2024-02-12: 2400751.849999933
2024-02-13: 1637187.279999944
2024-02-14: 2909888.909999908
2024-02-15: 2928848.609999918
2024-02-16: 2679060.249999922
2024-02-17: 2312374.7899999036
2024-02-18: 2187241.4499999024
2024-02-19: 2080471.0499999125
2024-02-20: 2546112.629999959
2024-02-21: 2635096.6199999717
2024-02-22: 2952750.1499999287
2024-02-23: 2614497.559999943
2024-02-24: 2626553.7999999304
2024-02-25: 2336178.379999912
2024-02-26: 2398380.339999892
2024-02-27: 2768085.449999888
2024-02-28: 2819418.339999884
2024-02-29: 3129240.9699998614


## Join daily data

In [118]:
daily_trips_revenue = daily_trips.join(daily_revenue).map(
    lambda x: (x[0], x[1][0], x[1][1])
)
daily_trips_revenue.collect()

[(datetime.date(2024, 2, 4), 80983, 2237400.2499998766),
 (datetime.date(2024, 2, 13), 64354, 1637187.279999944),
 (datetime.date(2024, 2, 17), 91075, 2312374.7899999036),
 (datetime.date(2024, 2, 21), 96718, 2635096.6199999717),
 (datetime.date(2024, 2, 1), 102070, 2824015.919999942),
 (datetime.date(2024, 2, 8), 105306, 2960078.48999986),
 (datetime.date(2024, 2, 11), 85507, 2330352.6399999503),
 (datetime.date(2024, 2, 14), 108629, 2909888.909999908),
 (datetime.date(2024, 2, 16), 94776, 2679060.249999922),
 (datetime.date(2024, 2, 19), 70098, 2080471.0499999125),
 (datetime.date(2024, 2, 9), 101060, 2804011.5299998904),
 (datetime.date(2024, 2, 24), 103426, 2626553.7999999304),
 (datetime.date(2024, 2, 26), 79291, 2398380.339999892),
 (datetime.date(2024, 2, 6), 95328, 2580443.3999999897),
 (datetime.date(2024, 2, 15), 107069, 2928848.609999918),
 (datetime.date(2024, 2, 28), 100272, 2819418.339999884),
 (datetime.date(2024, 2, 18), 80718, 2187241.4499999024),
 (datetime.date(2024,

## Save Results

In [117]:
import os

base_dir = 'materials/data_engineering_course_materials/missions/W5/results/'
os.makedirs(base_dir, exist_ok=True)

# 기본적으로 partition으로 저장됨

# Save results as CSV
daily_trips_revenue = daily_trips_revenue.toDF(['date', 'count', 'revenue'])
# 파티션 1개로 저장하기
daily_trips_revenue.coalesce(1).write.csv(f'{base_dir}/daily_trips_revenues.csv', header=True, mode="overwrite")

# Save results as Parquet
stats_df = spark.createDataFrame([stats], ['total_trips', 'total_revenue', 'avg_distance'])
stats_df.coalesce(1).write.parquet(f'{base_dir}/stats.parquet', mode="overwrite")